# Modelo Machine Learning

## Importamos las librerias

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from surprise import Dataset
from surprise import Reader
from surprise.model_selection import train_test_split

In [2]:
reader = Reader()

## Cargamos nuestros datos resultandes del EDA

In [4]:
df1 = pd.read_csv("Machine_Learning_Data/Calificaciones.csv") # CALIFICACIONES

In [5]:
df2 = pd.read_csv("Machine_Learning_Data/Titulos.csv") # NOMBRES DE PELICULAS

### Seleccionamos la informacion

In [7]:
# Ya que es mucha info la que se esta manejando seleccionare solo una parte de los datos
# Usare los datos del dataframe de calificaciones que exista cada 4 filas
data = Dataset.load_from_df(df1[['userId', 'id', 'score']][::4], reader)

In [8]:
# Separamos nuestros datos
trainset, testset = train_test_split(data, test_size=.25)

In [9]:
# Usaremos un modelo de Singular Value Decomposition
from surprise import SVD
model = SVD()

### Entrenamiento

In [10]:
# Entrenamos el modelo
model.fit(trainset)

### Prediccion

In [11]:
# Predecimos
predictions = model.test(testset)

In [12]:
predictions[1]

Prediction(uid=60280, iid='as1643', r_ui=4.0, est=3.8693095776192195, details={'was_impossible': False})

In [13]:
# Hacemos una predicción al azar para usuario y película
model.predict(500,"as382")

Prediction(uid=500, iid='as382', r_ui=None, est=3.765571177300336, details={'was_impossible': False})

## Eleccion de usuario

In [17]:
# Tomaremos un usuario para hacerle una recomendación
usuario = 200
rating = 4   # Tomamos películas a las que haya calificado con mas de 3 estrellas
df_user = df1[(df1['userId'] == usuario) & (df1['score'] >= rating)]

In [19]:
df_user.head()

,userId,score,id
225004,200,4.0,as545
935886,200,4.0,as2262
2135789,200,4.0,as5161
2404706,200,4.0,as5810
2504650,200,4.5,as6052


In [20]:
df_user = df_user.reset_index(drop=True)

In [21]:
df_user = df_user.merge(df2, on="id")
df_user.head()

,userId,score,id,title
0,200,4.0,as545,the mandela effect
1,200,4.0,as2262,hannie caulder
2,200,4.0,as5161,one step behind the seraphim
3,200,4.0,as5810,lea & pop - baby shark & more kids songs
4,200,4.5,as6052,pinkfong! phonics songs
5,200,4.0,as6517,the isle
6,200,5.0,as6668,the hackers
7,200,4.0,as6791,the ring-a-tangs - baby shark & more puppet ki...
8,200,4.5,as7203,inga enna solluthu
9,200,4.0,as7509,the california no


In [22]:
recomendaciones_usuario = df2.iloc[:4499].copy()
print(recomendaciones_usuario.shape)
recomendaciones_usuario.head()

(4499, 2)


,id,title
0,as1,the grand seduction
1,as2,take care good night
2,as3,secrets of deception
3,as4,pink: staying true
4,as5,monster maker


In [23]:
# Debemos extraer las películas que ya ha visto

usuario_vistas = df1[df1['userId'] == usuario]
print(usuario_vistas.shape)
usuario_vistas.head(15)

(67, 3)


,userId,score,id
225004,200,4.0,as545
935886,200,4.0,as2262
2064586,200,2.5,as4989
2135789,200,4.0,as5161
2270885,200,1.0,as5488
2404706,200,4.0,as5810
2504650,200,4.5,as6052
2606521,200,3.0,as6299
2696321,200,4.0,as6517
2758756,200,5.0,as6668


## Peliculas que se recomiendan

In [24]:
# Recomendamos

recomendaciones_usuario['Estimate_Score'] = recomendaciones_usuario['id'].apply(lambda x: model.predict(usuario, x).est)

In [25]:
recomendaciones_usuario = recomendaciones_usuario.sort_values('Estimate_Score', ascending=False)
recomendaciones_usuario.head()

,id,title,Estimate_Score
3359,as3360,across the pacific,4.265672
4179,as4180,15-minute full body balance & tone 7.0 workout...,4.230867
3403,as3404,a mother's nightmare,4.217351
3511,as3512,16 minute chakra garden meditation,4.216472
2329,as2330,go buster - baby bus cartoons,4.211866


## Evaluacion

In [26]:
from surprise import accuracy

accuracy.rmse(predictions)

RMSE: 0.9945


0.994496274663754